In [1]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import  LabelEncoder
from tqdm.auto import tqdm
import random
import ast
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import json
import torch
import torch.nn.functional as F
from ast import literal_eval
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import optim
from sklearn.model_selection import train_test_split
import torch.nn as nn

import inflect

p = inflect.engine()


from nltk.tokenize import word_tokenize


from nltk.tokenize.treebank import TreebankWordDetokenizer

/Users/tanmayagarwal/opt/miniconda3/envs/BERT/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
features = pd.read_csv("data/nbme-score-clinical-patient-notes/features.csv")
patient_notes = pd.read_csv("data/nbme-score-clinical-patient-notes/patient_notes.csv")

In [6]:
train_data = pd.read_csv("data/nbme-score-clinical-patient-notes/train.csv")

In [7]:
features

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
...,...,...,...
138,912,9,Family-history-of-migraines
139,913,9,Female
140,914,9,Photophobia
141,915,9,No-known-illness-contacts


In [8]:
patient_notes

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...
...,...,...,...
42141,95330,9,Ms. Madden is a 20 yo female presenting w/ the...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...
42144,95333,9,Stephanie madden is a 20 year old woman compla...


In [10]:
merged_train_data = train_data.merge(patient_notes,on=['case_num','pn_num']).merge(features,on=['case_num','feature_num'])


In [11]:
merged_train_data

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...
1,00041_000,0,41,0,[],[],17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,Family-history-of-MI-OR-Family-history-of-myoc...
2,00046_000,0,46,0,['father: heart attack'],['824 844'],Mr. Cleveland is a 17yo M who was consented by...,Family-history-of-MI-OR-Family-history-of-myoc...
3,00082_000,0,82,0,['Father MI'],['622 631'],17 yo M w/ no cardiac or arrhythmia PMH presen...,Family-history-of-MI-OR-Family-history-of-myoc...
4,00100_000,0,100,0,['Dad-MI'],['735 741'],HPI: Dillon Cleveland is an otherwise healthy ...,Family-history-of-MI-OR-Family-history-of-myoc...
...,...,...,...,...,...,...,...,...
14295,95145_916,9,95145,916,['subjective fever'],['169 185'],Pt is 20 yo F w headache since yesterday morni...,Subjective-fever
14296,95228_916,9,95228,916,[],[],"20 F no PMH, lives w/ roommate in apartment ha...",Subjective-fever
14297,95243_916,9,95243,916,['feels warm'],['376 386'],20 y/o F with no PMH is presenting with 1 day ...,Subjective-fever
14298,95330_916,9,95330,916,['Felt warm'],['358 367'],Ms. Madden is a 20 yo female presenting w/ the...,Subjective-fever


In [126]:
def convert_special_char_word(num_year_old, special_char):
    words = num_year_old.split(special_char)
    p = inflect.engine()
    for word in words:
        if word.isdigit():
            number = p.number_to_words(word)
            idx = words.index(word)
            words[idx] =  number 
            digit_found = True
    return (special_char).join(words)

In [130]:
def extract_numbers_from_paragraghs(para):
    words = word_tokenize(para)
    new_para = []
    list_special_char_check = ['-', ',', '/']
    for word in words:
        for special_char in list_special_char_check:
            if special_char in word:
                word= convert_special_char_word(word, special_char)
                
                
        if word.isdigit():
            word = p.number_to_words(word)
            
        new_para.append(word)
        
    return ' '.join(new_para)
                      


In [140]:
extract_numbers_from_paragraghs(patient_notes['pn_history'].iloc[4])

"17yo male with no pmh here for evaluation of palpitations . States for the last three-4mo he has felt that his heart with intermittently `` beat out of his chest , '' with some associated difficulty catching his breath . States that the most recent event was two days ago , and during activity at a soccer game . He does not seem to note any specific precipitatinig factors at this time . He also states that he feels as if he will faint during these events , but has not lost consciousness at any point . Furthermore , he does endorse theses attacks occuring one-two times a month and peak at four mins . He denies any stressors at home . ROS : denies weight loss , fevers , recnet illness , change in bowel habits . PMH : negative , PSH negative , FHX mom with thyroid disorder , dad with heart condition and MI at 52yo . SHX no tobacco , ETOH on weekends , Marijuana tried a month ago . Med : is taking some of roommates Adderoll intermittently ( last was two days ago prior to event ) . KNDA"

In [141]:
patient_notes['pn_history'].iloc[4]

'17yo male with no pmh here for evaluation of palpitations. States for the last 3-4mo he has felt that his heart with intermittently "beat out of his chest," with some associated difficulty catching his breath. States that the most recent event was 2 days ago, and during activity at a soccer game. He does not seem to note any specific precipitatinig factors at this time. He also states that he feels as if he will faint during these events, but has not lost consciousness at any point. Furthermore, he does endorse theses attacks occuring 1-2 times a month and peak at 4 mins. He denies any stressors at home. ROS: denies weight loss, fevers, recnet illness, change in bowel habits. PMH: negative, PSH negative, FHX mom with thyroid disorder, dad with heart condition and MI at 52yo. SHX no tobacco, ETOH on weekends, Marijuana tried a month ago. Med: is taking some of roommates Adderoll intermittently (last was 2 days ago prior to event). KNDA'

In [82]:
extract_numbers_from_paragraghs(patient_notes['pn_history'].iloc[0])

seventeen-year-old
two-three
two
three-four
five-six


In [67]:
word_tokenize(patient_notes['pn_history'].iloc[0])

['17-year-old',
 'male',
 ',',
 'has',
 'come',
 'to',
 'the',
 'student',
 'health',
 'clinic',
 'complaining',
 'of',
 'heart',
 'pounding',
 '.',
 'Mr.',
 'Cleveland',
 "'s",
 'mother',
 'has',
 'given',
 'verbal',
 'consent',
 'for',
 'a',
 'history',
 ',',
 'physical',
 'examination',
 ',',
 'and',
 'treatment',
 '-began',
 '2-3',
 'months',
 'ago',
 ',',
 'sudden',
 ',',
 'intermittent',
 'for',
 '2',
 'days',
 '(',
 'lasting',
 '3-4',
 'min',
 ')',
 ',',
 'worsening',
 ',',
 'non-allev/aggrav',
 '-associated',
 'with',
 'dispnea',
 'on',
 'exersion',
 'and',
 'rest',
 ',',
 'stressed',
 'out',
 'about',
 'school',
 '-reports',
 'fe',
 'feels',
 'like',
 'his',
 'heart',
 'is',
 'jumping',
 'out',
 'of',
 'his',
 'chest',
 '-ros',
 ':',
 'denies',
 'chest',
 'pain',
 ',',
 'dyaphoresis',
 ',',
 'wt',
 'loss',
 ',',
 'chills',
 ',',
 'fever',
 ',',
 'nausea',
 ',',
 'vomiting',
 ',',
 'pedal',
 'edeam',
 '-pmh',
 ':',
 'non',
 ',',
 'meds',
 ':',
 'aderol',
 '(',
 'from',
 'a',
 '

In [59]:
word_tokenize("17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"             )

['17-year-old',
 'male',
 ',',
 'has',
 'come',
 'to',
 'the',
 'student',
 'health',
 'clinic',
 'complaining',
 'of',
 'heart',
 'pounding',
 '.',
 'Mr.',
 'Cleveland',
 "'s",
 'mother',
 'has',
 'given',
 'verbal',
 'consent',
 'for',
 'a',
 'history',
 ',',
 'physical',
 'examination',
 ',',
 'and',
 'treatment',
 '-began',
 '2-3',
 'months',
 'ago',
 ',',
 'sudden',
 ',',
 'intermittent',
 'for',
 '2',
 'days',
 '(',
 'lasting',
 '3-4',
 'min',
 ')',
 ',',
 'worsening',
 ',',
 'non-allev/aggrav',
 '-associated',
 'with',
 'dispnea',
 'on',
 'exersion',
 'and',
 'rest',
 ',',
 'stressed',
 'out',
 'about',
 'school',
 '-reports',
 'fe',
 'feels',
 'like',
 'his',
 'heart',
 'is',
 'jumping',
 'out',
 'of',
 'his',
 'chest',
 '-ros',
 ':',
 'denies',
 'chest',
 'pain',
 ',',
 'dyaphoresis',
 ',',
 'wt',
 'loss',
 ',',
 'chills',
 ',',
 'fever',
 ',',
 'nausea',
 ',',
 'vomiting',
 ',',
 'pedal',
 'edeam',
 '-pmh',
 ':',
 'non',
 ',',
 'meds',
 ':',
 'aderol',
 '(',
 'from',
 'a',
 '

In [24]:
patient_notes['pn_history'].iloc[0]

"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"

In [ ]:
yo
y.o.

In [14]:
print(merged_train_data.pn_history.iloc[10])

17 yo M complains of heart pounding since 2-3 months. He had 5-6 episodes in the past 2-3 months. It is progressively worsening. No specific triggers noticed when these happen. No sweating/chest pain/N/B, SOB. The last episode was severe that he thought he would passout. Denies headaches/ blurry vision/cough/swelling of legs
ROS- negative except as above
PMH/Allergies/PSH- none
Meds- takes adderall since college 
FH- Mother has thyroid issues, Father has heart problem
SH-smoked pot once, nonsmoker, Etoh on weekends occasionally, college student, sexually active with girlfriend
